In [1]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import LinearSVC
from sklearn.model_selection import train_test_split

In [4]:
df = pd.read_excel('data/FN_Training_Set.xlsx')

In [19]:
df['Content'] = df['Content'].astype(str)
df['Content Title'] = df['Content Title'].astype(str)

In [77]:
df['len_words'] = df['Content'].apply(lambda t: len(t.split()))
df['len_words_title'] = df['Content Title'].apply(lambda t: len(t.split()))

df['len_chars'] = df['Content'].apply(len)
df['len_chars_title'] = df['Content Title'].apply(len)

df['len_symbols'] = df['Content'].apply(lambda t: sum([c in ['$.!;#?:-+@%^&*(),'] for c in t]))
df['len_symbols_title'] = df['Content Title'].apply(lambda t: sum([c in ['$.!;#?:-+@%^&*(),'] for c in t]))

df['len_capitals'] = df['Content'].apply(lambda t: sum([str.isupper(c) for c in t]))
df['len_capitals_title'] = df['Content Title'].apply(lambda t: sum([str.isupper(c) for c in t]))

df['fraction_capitals'] = (df['len_capitals'] + 1) / (df['len_chars'] + 1)
df['fraction_capitals_title'] = (df['len_capitals_title'] + 1) / (df['len_chars_title'] + 1)


In [106]:
from urllib import parse

# False positives exist
def extract_urls(text):
    """Return a list of urls from a text string."""
    out = []
    for word in text.split(' '):
        thing = parse.urlparse(word.strip())
        if thing.scheme:
            out.append(word)
    return out

In [132]:
df['len_url'] = df.Content.apply(lambda t: len(extract_urls(t)))

In [133]:
df

,fake_news_score,click_bait_score,Content Title,Content Url,Content Published Time,Content,len_words,len_words_title,len_chars,len_chars_title,len_symbols,len_symbols_title,len_capitals,len_capitals_title,fraction_capitals,fraction_capitals_title,len_url
0,1,1,Камил Хабиб: ДАЕШ и „Ан Нусра” са нов терорист...,http://a-specto.bg/kamil-habib-daesh-i-nusra-s...,2017-05-17 18:35:00,"Интервю на Десислава Пътева с Камил Хабиб, дек...",1169,10,7345,58,0,0,201,8,0.027498,0.152542,0
1,1,1,ЧЕТИРИТЕ ТИПА ЖЕНИ СПОРЕД ИЗТОЧНИЯ КАЛЕНДАР,http://novini24online.com/jba/113526,2017-05-17 11:48:00,"Романтичната жена Това са жените, родени в год...",1744,6,11287,43,0,0,340,38,0.030209,0.886364,0
2,1,1,Арт - Людмила Ким почита премиера за Ванга,http://paper.standartnews.com/bg/article.php?a...,2017-05-16 21:36:00,"Людмила Ким, най-популярната руска лечителка и...",225,8,1472,42,0,0,46,4,0.031908,0.116279,0
3,3,3,Истина или не ! Истина ли е дали Бойко е гей и...,http://fakenews.bg/Истина-или-не-Истина-ли-е-д...,2017-05-16 19:40:00,Истина или не ! Истина ли е дали Бойко е гей и...,672,17,3727,72,0,0,139,6,0.037554,0.095890,0
4,1,1,"Видеото на Чарли Чаплин, в което няма да се см...",http://blife.eu/видеото-чарли-чаплин-което-ням...,2017-05-16 13:41:00,"Свикнали сме Чарли Чаплин да ни разсмива, да х...",601,10,3635,51,0,0,77,3,0.021452,0.076923,0
5,1,1,"""Реквием за Ванга от Людмила Ким"" с безплатна ...",http://bnr.bg/post/100831443/vanga-film,2017-05-16 12:55:00,"Руско-българският документален филм ""Реквием з...",282,12,1659,73,0,0,33,4,0.020482,0.067568,0
6,1,1,"""Реквием за Ванга от Людмила Ким"" с безплатна ...",http://bnr.bg/horizont/post/100831443/vanga-film,2017-05-16 12:55:00,"Руско-българският документален филм ""Реквием з...",282,12,1659,73,0,0,33,4,0.020482,0.067568,0
7,3,3,Оракулът от село Граница Василка Анакиева: Пол...,https://skafeto.com/любопитно/оракулът-от-село...,2017-05-16 12:36:00,Оракулът от село Граница Василка Анакиева прид...,304,18,1955,104,0,0,65,7,0.033742,0.076190,0
8,3,3,"Българските илюминати, незнаещи, че са пионки ...",http://bezistena.blog.bg/history/2017/05/15/by...,2017-05-15 21:18:00,Все повече и повече директни или съвсем леко з...,2080,16,13405,96,0,0,1946,2,0.145233,0.030928,0
9,3,3,Тайните апокалиптични скривалища на световния ...,http://gudelnews.com/index.php?option=com_cont...,2017-05-15 15:12:00,"През 2012-2013 г. световният елит, според няко...",519,8,3449,59,0,0,70,1,0.020580,0.033333,0


In [134]:
df_train, df_test = train_test_split(df, test_size=0.1, random_state=999)

In [135]:
df.columns

Index(['fake_news_score', 'click_bait_score', 'Content Title', 'Content Url',
       'Content Published Time', 'Content', 'len_words', 'len_words_title',
       'len_chars', 'len_chars_title', 'len_symbols', 'len_symbols_title',
       'len_capitals', 'len_capitals_title', 'fraction_capitals',
       'fraction_capitals_title', 'len_url'],
      dtype='object')

In [136]:
columns = ['len_words', 'len_words_title',
           'len_chars', 'len_chars_title', 'len_symbols', 'len_symbols_title',
           'len_capitals', 'len_capitals_title', 'fraction_capitals',
           'fraction_capitals_title', 'len_url']

In [139]:
from sklearn.ensemble import RandomForestClassifier
clf = RandomForestClassifier(50, random_state=29910)
clf.fit(df_train[columns], df_train['fake_news_score'])
clf.score(df_test[columns], df_test['fake_news_score'])

0.72340425531914898

In [140]:
clf = LinearSVC(random_state=9999)
clf.fit(df_train[columns], df_train['fake_news_score'])
clf.score(df_test[columns], df_test['fake_news_score'])

0.67375886524822692